In [1]:
# generic imports
import numpy as np
import pandas as pd
import cvxpy as cp
import math

In [2]:
# initial time step and final time under control
# 100,000 seconds is a little over a day
timeStep = 100
numSteps = 1000

# inital x sattelite (x,y,z,dx,dy,dz)
x_0 = np.array([0,1,0,0,0,0]).reshape((6,1))

# phi matrix given time and beginning time.
def phi(t_i,t_0, n=(2*np.pi) / 86400):
    t = t_i - t_0
    return np.array([[4 - 3*math.cos(n*t), 0, 0, (1/n) * math.sin(n*t), (2/n) * (1 - math.cos(n*t)),0],
                 [6*(math.sin(n*t)-n*t),1, 0, -1*(2/n)*( 1 - math.cos(n*t)), (1/n)*(4*math.sin(n*t) - 3*n*t),0],
                 [0,0,math.cos(n*t),0,0,(1/n)*math.sin(n*t)],
                 [3*n*math.sin(n*t),0,0,math.cos(n*t),2*math.sin(n*t),0],
                 [-6*n*(1-math.cos(n*t)),0,0,-2*math.sin(n*t), 4*math.cos(n*t)-3, 0], 
                 [0,0,-1*n*math.sin(n*t), 0, 0, math.cos(n*t)]])

# find the specific velocity pebetuation that we want for a given time step.
def delt(u,i,delta):
    return np.array([[0],[0],[0],[u[i][0]*delta],[u[i][1]*delta],[u[i][2]*delta]])

# move from x_0 to x_i
def move(x_0, u, numSteps, delta):
    x_i = phi(numSteps*delta, 0) @ x_0
    for j in range(1, numSteps):
        x_i = x_i + phi(numSteps*delta, j*delta) @ delt(u,j,delta)

    return x_i

def moveOneStep(x_0, u, delta, i):
    x_i = x_0 + phi(delta, 0) @ delt(u,i,delta)
    return x_i

# generate massive PHI:
def generatePhi(x_0, numSteps, delta):
    # phi is 6x6 with x_0 on diagonal
    PHI = np.eye(6) * x_0
    
    for i in range(0, numSteps):
        PHI = np.append(PHI, phi(numSteps*delta, i*delta))
    return PHI.reshape((6*(numSteps+1),6))

In [3]:
# total PHI. Is 6006 x 6
PHI = generatePhi(x_0, numSteps, timeStep)
# generate total u
u = np.random.rand(numSteps+1,6).reshape((numSteps+1)*6,1)


print("PHI Shape: " + str(PHI.T.shape))
print("u Shape: " + str(u.shape))

# u is of shape (1001,6) -> (6006, 1) 
PHI.T @ u

A = []
for i in range(0, (numSteps+1)):
    A.append([1]*3 + [0]*3)
A = np.array(A).reshape((numSteps+1)*6,1)
print("A Shape: " + str(A.shape))

# opposite of A
B = []
for i in range(0, (numSteps+1)):
    B.append([0]*3 + [1]*3)
B = np.array(B).reshape((numSteps+1)*6,1)
print("B Shape: " + str(B.shape))

PHI Shape: (6, 6006)
u Shape: (6006, 1)
A Shape: (6006, 1)
B Shape: (6006, 1)


In [4]:
from tabnanny import verbose

maxProp = 0.1

F = np.ones((6*numSteps+1,1)) * maxProp
Z = np.zeros((6*numSteps+1,1))
u = cp.Variable( ((numSteps+1)*6,1)) 
objective = cp.Minimize( np.ones(6) @ (PHI.T @ u) ) 
constraints = [A.T @ u == 0]
constraints += [B >= u ]

prob = cp.Problem(objective, constraints)  # Form problem.
prob.solve(verbose=True)  # Solve problem.


print("Thrust Vectors at Final Time Step")
#print(u.value)
print("Final Position")
#print( PHI.T @ u )

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Oct 30 11:54:15 PM: Your problem has 6006 variables, 2 constraints, and 0 parameters.
(CVXPY) Oct 30 11:54:15 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 30 11:54:15 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 30 11:54:15 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 30 11:54:15 PM: Compiling problem (target solver=ECOS).
(CVXPY) Oct 30 11:54:15 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

In [ ]:
# plot trajectory
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

x_trajectory = [x_0[0][0]]
y_trajectory = [x_0[1][0]]
z_trajectory = [x_0[2][0]]

x_i = x_0
# get x,y,z values for plotting
for i in range(1,numSteps+1):
    #x_i = moveOneStep(x_i, u.value, timeStep, i)
    x_i = move(x_0, u.value, i, timeStep)
    # get first three elements of x_i
    x_trajectory.append(x_i[0][0])
    y_trajectory.append(x_i[1][0])
    z_trajectory.append(x_i[2][0])

print("Final Position")
print((x_trajectory[-1], y_trajectory[-1], z_trajectory[-1]))
# print sizes

# plot trajectory
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot(x_trajectory, y_trajectory, z_trajectory)
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

fig = plt.figure()
ax = fig.add_subplot()
ax.plot(y_trajectory, x_trajectory)
ax.set_xlabel('Y Label')
ax.set_ylabel('X Label')

# plot usage of thrust magnitude at each time step
thrust = [ math.sqrt(u.value[i][0]**2 + u.value[i][1]**2 + u.value[i][2]**2) for i in range(numSteps)]
#print(thrust)
plt.figure()
plt.plot(thrust)
plt.xlabel('time')

plt.show()

TypeError: 'NoneType' object is not subscriptable